In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/500-greatest-songs-of-all-time/Top 500 Songs.csv


# Import libraries

In [34]:
import pandas as pd
import numpy as np

# Machine learning library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Reading file

In [32]:
file = "/kaggle/input/500-greatest-songs-of-all-time/Top 500 Songs.csv"
df_songs = pd.read_csv(file, delimiter=',', encoding = "ISO-8859-1")

In [30]:
df_songs['description']

0      "I wrote it. I didn't fail. It was straight," ...
1      "It's the riff heard round the world," says St...
2      John Lennon wrote "Imagine," his greatest musi...
3      "What's Going On" is an exquisite plea for pea...
4      Otis Redding wrote "Respect" and recorded it f...
                             ...                        
495    This seven-minute, two-chord track spiked out ...
496    Morton found the inspiration for this song at ...
497    Toots and the Maytals were already reggae star...
498    "It's just about people and what they're expec...
499    Late one night, while Sonny and Cher were livi...
Name: description, Length: 500, dtype: object

# Filter out all the filler words in english. 

In [33]:
tfidf = TfidfVectorizer(stop_words = "english")
df_songs['description'] = df_songs['description'].fillna("")
tfidfMatrix = tfidf.fit_transform(df_songs['description'])

# Building a linear kernel

I decided to use a linear kernel because there are many different features/keywords listed in the description for each song. The data is also linearly seperable, which makes a linear kernel much more useful.

In [39]:
cosSim = linear_kernel(tfidfMatrix, tfidfMatrix)

# Indexing each song

In [40]:
indices = pd.Series(df_songs.index, index=df_songs['title']).drop_duplicates()
indices

title
Like a Rolling Stone                0
(I Can't Get No) Satisfaction'      1
Imagine                             2
What's Going On                     3
Respect                             4
                                 ... 
Heroin                            495
Leader of the Pack                496
Pressure Drop                     497
Come As You Are                   498
I Got You Babe                    499
Length: 500, dtype: int64

# GetRecommendations function: Finds the 10 best matching score for the given movie



In [43]:
def getRecommendations(title, cosSim = cosSim):
    index = indices[title]
    simScores = enumerate(cosSim[index]) # Scores from 0.0 - 1.0; higher score = better match
    simScores = sorted(simScores, key=lambda x:x[1], reverse=True) # Sorting scpres
    simScores = simScores[1:11] # 10 scores 
    
    simIndex = [i[0] for i in simScores]
    print(df_songs['title'].iloc[simIndex])
    
getRecommendations("Imagine") 

7                                  Hey Jude
74                Strawberry Fields Forever
124               Strawberry Fields Forever
380                    All You Need is Love
22                               In My Life
164                        Please Please Me
238                              In My Room
27                        A Day in the Life
67     Norwegian Wood (This Bird Has Flown)
117    Norwegian Wood (This Bird Has Flown)
Name: title, dtype: object
